# Amazon SageMaker Batch Transform: Associate prediction results with their corresponding input records


In [2]:
# !pip3 install -U sagemaker

In [3]:
import os
import boto3
import sagemaker
import pandas as pd
import numpy as np

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

s3 = boto3.client("s3")
bucket = sess.default_bucket()
prefix = "demo-sagemaker-xgboost-Diabetes-prediction"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


---

## Training job and model creation

The below cell uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick off the training job using both our training set and validation set. Not that the objective is set to 'binary:logistic' which trains a model to output a probability between 0 and 1 (here the probability of a tumor being malignant).

In [4]:
%%time
from time import gmtime, strftime

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=boto3.Session().region_name, version="1.7-1"
)

sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sess,
)

sm_estimator.set_hyperparameters(
    objective="binary:logistic",
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    num_round=100,
)

train_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validation".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

# Start training by calling the fit method in the estimator
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker:Creating training-job with name: xgb-2024-06-16-13-27-27


2024-06-16 13:27:28 Starting - Starting the training job...
2024-06-16 13:27:43 Starting - Preparing the instances for training...
2024-06-16 13:28:14 Downloading - Downloading input data...
2024-06-16 13:28:44 Downloading - Downloading the training image...
2024-06-16 13:29:19 Training - Training image download completed. Training in progress...[2024-06-16 13:29:30.827 ip-10-0-147-167.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-06-16 13:29:30.849 ip-10-0-147-167.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-06-16:13:29:31:INFO] Imported framework sagemaker_xgboost_container.training
[2024-06-16:13:29:31:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-06-16:13:29:31:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-16:13:29:31:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-06-16:13:29:31:INFO] Determined 0 GPU(s) available on the inst

In [5]:
sm_estimator.model_data

's3://sagemaker-us-east-1-741135916424/demo-sagemaker-xgboost-Diabetes-prediction/output/xgb-2024-06-16-13-27-27/xgb-2024-06-16-13-27-27/output/model.tar.gz'

---

## Batch Transform

In SageMaker Batch Transform, we introduced 3 new attributes - __input_filter__, __join_source__ and __output_filter__. In the below cell, we use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick-off several Batch Transform jobs using different configurations of these 3 new attributes. Please refer to [this page](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to use them.




#### 1. Create a transform job with the default configurations
Let's first skip these 3 new attributes and inspect the inference results. We'll use it as a baseline to compare to the results with data processing.

In [6]:
%%time

sm_transformer = sm_estimator.transformer(1, "ml.m5.xlarge")
batch_file = "batch.csv"

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)  # use input data without ID column
sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")
sm_transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-06-16-13-30-10-455
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-16-13-30-11-292


....................................
[2024-06-16:13:36:16:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-16:13:36:16:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-16:13:36:16:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-06-16 13:36:16 +0000] [19] [INFO] Startin

Let's inspect the output of the Batch Transform job in S3. It should show the list probabilities of tumors being malignant.

In [7]:
import re

def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [8]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0
0,0.724473
1,0.539312
2,0.292787
3,0.230532
4,0.755518
5,0.090598
6,0.371944
7,0.619587


#### 2. Join the input and the prediction results 
Now, let's associate the prediction results with their corresponding input records. We can also use the __input_filter__ to exclude the ID column easily and there's no need to have a separate file in S3.

* Set __input_filter__ to "$[1:]": indicates that we are excluding column 0 (the 'ID') before processing the inferences and keeping everything from column 1 to the last column (all the features or predictors)  
  
  
* Set __join_source__ to "Input": indicates our desire to join the input data with the inference results  

* Leave __output_filter__ to default ('$'), indicating that the joined input and inference results be will saved as output.

In [9]:
# content_type / accept and split_type / assemble_with are required to use IO joining feature
sm_transformer.assemble_with = "Line"
sm_transformer.accept = "text/csv"

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)  # use input data with ID column cause InputFilter will filter it out
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-16-13-36-45-603


................................[2024-06-16:13:42:07:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-16:13:42:07:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-16:13:42:07:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-06-16 13:42:07 +0000] [20] [INFO] Starting gun

Let's inspect the output of the Batch Transform job in S3. It should show the list of tumors identified by their original feature columns and their corresponding probabilities of being malignant.

In [10]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1.0,1.0,1.0,29.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,4.0,30.0,5.0,0.0,0.0,6.0,4.0,3.0,0.819771
1,1.0,1.0,1.0,30.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,11.0,6.0,7.0,0.244050
2,0.0,0.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,10.0,5.0,6.0,0.088419
3,0.0,1.0,1.0,23.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,3.0,27.0,29.0,0.0,0.0,8.0,6.0,6.0,0.572744
4,1.0,0.0,1.0,33.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,3.0,0.0,0.0,0.0,1.0,10.0,6.0,4.0,0.340352
5,0.0,0.0,1.0,23.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,2.0,0.0,0.0,0.0,1.0,5.0,4.0,7.0,0.140086
6,1.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,10.0,5.0,8.0,0.234666
7,1.0,0.0,1.0,42.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,2.0,1.0,0.0,0.0,0.0,10.0,4.0,5.0,0.119529


#### 3. Update the output filter to keep only ID and prediction results
Let's change __output_filter__ to "$[0,-1]", indicating that when presenting the output, we only want to keep column 0 (the 'ID') and the last column (the inference result i.e. the probability of a given tumor to be malignant)

In [11]:
# start another transform job
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
    output_filter="$[0,-1]",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-16-13-42-49-937


................................[2024-06-16:13:48:06:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-16:13:48:06:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-16:13:48:06:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-06-16 13:48:06 +0000] [19] [INFO] Starting gun

Now, let's inspect the output of the Batch Transform job in S3 again. It should show 2 columns: the ID and their corresponding probabilities of being malignant.

In [12]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1
0,1.0,0.819771
1,1.0,0.244050
2,0.0,0.088419
3,0.0,0.572744
4,1.0,0.340352
5,0.0,0.140086
6,1.0,0.234666
7,1.0,0.119529


create_model(role=role, image_uri=XGBOOST_IMAGE)In summary, we can use newly introduced 3 attributes - __input_filter__, __join_source__, __output_filter__ to 
1. Filter / select useful features from the input dataset. e.g. exclude ID columns.
2. Associate the prediction results with their corresponding input records.
3. Filter the original or joined results before saving to S3. e.g. keep ID and probability columns only.

## Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [13]:
# Initialize the SageMaker client
sagemaker = boto3.client('sagemaker')

# List all training jobs
response = sagemaker.list_training_jobs()

# Print out the names of the training jobs
i = 0
model_name = ""
for job in response['TrainingJobSummaries']:
    if i == 0:
        model_name = job['TrainingJobName']
    i += 1
    print(job['TrainingJobName'])

xgb-2024-06-16-13-27-27
xgb-2024-06-15-23-45-14
pipelines-49mz8d35jsio-DiabetesTrain-NFBTBWa19W
pipelines-x1c300xbwm4g-DiabetesTrain-hZw07eXbjt
pipelines-cxsii09n2x5n-DiabetesTrain-LhsvUOcFLs
pipelines-43o5mtms55ka-DiabetesTrain-YtzqxK4Ju9
pipelines-8ai5z9w8lezh-DiabetesTrain-0M42OrOKeT
pipelines-494xx59mw9o9-DiabetesTrain-mpEyv0ocIY
pipelines-c25fumuxfkb0-DiabetesTrain-KjzKHaYjZL
pipelines-weq3gmq3drjh-DiabetesTrain-oYGKBKyELV


In [14]:
print(model_name)

info = sagemaker.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}

# Save our model to the Sagemaker Model Registry
create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

xgb-2024-06-16-13-27-27
arn:aws:sagemaker:us-east-1:741135916424:model/xgb-2024-06-16-13-27-27


In [15]:
# Inspect Training Job Details
info

{'TrainingJobName': 'xgb-2024-06-16-13-27-27',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:741135916424:training-job/xgb-2024-06-16-13-27-27',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-741135916424/demo-sagemaker-xgboost-Diabetes-prediction/output/xgb-2024-06-16-13-27-27/xgb-2024-06-16-13-27-27/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'binary:logistic',
  'subsample': '0.8',
  'verbosity': '0'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:aucpr',
    'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\\.?[0

In [16]:
# Create Endpoint Configuration
# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'lab4-1-endpoint-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            
                            
instance_type = 'ml.m5.xlarge'

endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

Created EndpointConfig: arn:aws:sagemaker:us-east-1:741135916424:endpoint-config/lab4-1-endpoint-config2024-06-16-13-48-55


In [17]:
# Deploy our model to real-time endpoint

endpoint_name = 'diabetes-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            


create_endpoint_response = sagemaker.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [18]:
from time import sleep

# Wait for endpoint to spin up
sagemaker.describe_endpoint(EndpointName=endpoint_name)

while True:
    print("Getting Job Status")
    res = sagemaker.describe_endpoint(EndpointName=endpoint_name)
    state = res["EndpointStatus"]
    
    if state == "InService":
        print("Endpoint in Service")
        break
    elif state == "Creating":
        print("Endpoint still creating...")
        sleep(60)
    else:
        print("Endpoint Creation Error - Check Sagemaker Console")
        break

Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint in Service


In [19]:
# Assuming 'data_batch' is a DataFrame, select the first 17 columns to match the model's expected input size
filename = "batch.csv"
data_batch = pd.read_csv(filename, header=None)

expected_feature_size = 17
data_batch_adjusted = data_batch.iloc[:, :expected_feature_size]

# Convert the adjusted DataFrame to CSV format without headers and indices
csv_input = data_batch_adjusted.to_csv(header=None, index=False).strip('\n')

# Initialize the SageMaker runtime client
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

# Invoke the endpoint with the adjusted input data
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=csv_input
)

# Read and decode the response
print(response['Body'].read().decode('utf-8'))


0.7007070183753967
0.254537433385849
0.04548318684101105
0.37487077713012695
0.17830443382263184
0.035933930426836014
0.3030657470226288
0.2835434377193451
0.7701780796051025
0.17341220378875732
0.03292294591665268
0.018298612907528877
0.25612011551856995
0.6835505962371826
0.4982294738292694
0.17960621416568756
0.8206015229225159
0.50139981508255
0.5406862497329712
0.8174253702163696
0.029853234067559242
0.8889501094818115
0.019644329324364662
0.8734901547431946
0.30457696318626404
0.858623743057251
0.07040242105722427
0.5576705932617188
0.10081296414136887
0.7555670738220215
0.015691418200731277
0.026300737634301186
0.11447641253471375
0.36658400297164917
0.025889910757541656
0.0938863456249237
0.2624702453613281
0.7463206648826599
0.12447308748960495
0.02157760038971901
0.9387983083724976
0.32235032320022583
0.06250089406967163
0.028294043615460396
0.3935871422290802
0.877591073513031
0.01597164012491703
0.8916125297546387
0.33642861247062683
0.05212823301553726
0.2543732821941376
0

In [20]:
# Invoke Endpoint

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
                            EndpointName=endpoint_name,
                            ContentType='text/csv',
                            Body=data_batch.to_csv(header=None, index=False).strip('\n').split('\n')[0]
                            )
print(response['Body'].read().decode('utf-8'))

0.7244731783866882



In [21]:
# Examine Response Body

response

{'ResponseMetadata': {'RequestId': '0a5b739a-dd3b-4d63-9944-ed6ca8516b5e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0a5b739a-dd3b-4d63-9944-ed6ca8516b5e',
   'x-amzn-invoked-production-variant': 'variant1',
   'date': 'Sun, 16 Jun 2024 13:52:57 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '19',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'variant1',
 'Body': <botocore.response.StreamingBody at 0x7f447c3b5810>}

In [22]:
# Delete Endpoint
sagemaker.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '91978db9-1189-4610-b804-9a75d708bebe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '91978db9-1189-4610-b804-9a75d708bebe',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 16 Jun 2024 13:52:57 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)
